In [54]:

import sys

sys.path.append('../../code/')
import os
import json
from datetime import datetime
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

import igraph as ig
from collections import *
from __future__ import division

from load_data import load_citation_network_igraph, case_info

%load_ext autoreload
%autoreload 2
%matplotlib inline

data_dir = '../../data/'
court_name = 'scotus'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# this will be a little slow the first time you run it
G = load_citation_network_igraph(data_dir, court_name)

print 'loaded %s network with %d cases and %d edges' % (court_name, len(G.vs), len(G.es))

0 seconds for 250465 edges
loaded scotus network with 33248 cases and 250465 edges


In [3]:
print G.vs[0]

igraph.Vertex(<igraph.Graph object at 0x000000000B662138>,0,{'court': 'scotus', 'name': 100000, 'year': 1922})


In [4]:
G.vs[0]['court']

'scotus'

In [5]:
G.vs[0].indegree()

1

In [17]:
year_list = []
indegree_list = []
id_list = []
igraph_index_list = []
for vertex in G.vs():
    year_list.append(vertex['year'])
    indegree_list.append(vertex.indegree())
    id_list.append(vertex['name'])
    igraph_index_list.append(vertex.index)

tuple_list = zip(igraph_index_list, id_list, year_list, indegree_list)

### keys = all the years, values = metadata of all past cases

In [18]:
min_year = np.amin(year_list)
max_year = np.amax(year_list)

past_now_cases_dict = OrderedDict()
for i in range (min_year,max_year+1):
    past_now_cases_dict[i] = []

for case in tuple_list:
    past_now_cases_dict[case[2]].append(case)

for i in range (min_year+1,max_year+1):
    past_now_cases_dict[i].extend(past_now_cases_dict[i-1])
    past_now_cases_dict[i] = sorted(past_now_cases_dict[i], key=lambda tup: tup[3], reverse=True)
    
past_cases_dict = OrderedDict()
for i in range (min_year,max_year+1):
    past_cases_dict[i+1] = past_now_cases_dict[i]

In [25]:
past_cases_dict[1770]

[(18427, 84581, 1754, 2),
 (17365, 2381647, 1768, 1),
 (17380, 2381728, 1768, 1),
 (18431, 84586, 1764, 1),
 (18433, 84589, 1768, 0),
 (18479, 84643, 1766, 0),
 (17398, 2381788, 1764, 0),
 (18432, 84587, 1764, 0),
 (18430, 84585, 1763, 0),
 (18396, 810012, 1760, 0),
 (18429, 84583, 1760, 0),
 (18428, 84582, 1759, 0)]

### 2nd part: get the cases that case c actually cites

In [56]:
# G.neighbors(10, mode='OUT') are the cases that case c actually cites

#ranks = []
for key, value in past_cases_dict.iteritems():
    if key == G.vs[10]['year']:
        all_past_cases = value

print len(all_past_cases)
print all_past_cases[0]



ranks = [i+1 for i, v in enumerate(all_past_cases) if v[0] in G.neighbors(10, mode='OUT')]

print len(ranks)
print ranks

scores = []
# score_i = 1 - r_i/ |B_c| (where |B_c| is the number of cases that came before case c)
for some_rank in ranks:
    some_score = 1-some_rank/len(all_past_cases)
    scores.append(some_score)
    
final_score = sum(scores)

print len(scores)
print scores
print 
print final_score


15293
(29701, 96405, 1906, 608)
26
[39, 317, 384, 439, 619, 652, 771, 864, 1275, 1340, 1644, 1721, 1736, 1955, 2119, 2617, 3326, 4132, 4365, 6120, 6340, 6372, 7181, 7297, 8424, 9674]
26
[0.9974498136402276, 0.9792715621526189, 0.9748904727653175, 0.9712940561040999, 0.9595239652128424, 0.9573661152161119, 0.949584777349114, 0.9435035637219643, 0.9166285228535931, 0.9123782122539724, 0.8924998365265154, 0.8874648532008108, 0.8864840122932061, 0.8721637350421761, 0.8614398744523638, 0.8288759563198849, 0.7825148760870987, 0.7298110246518015, 0.7145752958870071, 0.5998169096972471, 0.5854312430523769, 0.5833387824494867, 0.5304387628326686, 0.5228535931471916, 0.4491597462891519, 0.36742300398875305]

20.6561825672


In [58]:
# G.neighbors(10, mode='OUT') are the cases that case c actually cites

#ranks = []

def calculate_score_for_case(case_index, past_cases_dict):
    for key, value in past_cases_dict.iteritems():
        if key == G.vs[case_index]['year']:
            all_past_cases = value

    ranks = [i+1 for i, v in enumerate(all_past_cases) if v[0] in G.neighbors(case_index, mode='OUT')]

    scores = []
    for some_rank in ranks:
        some_score = 1-some_rank/len(all_past_cases)
        scores.append(some_score)

    final_score = sum(scores)
    
    return final_score
    
calculate_score_for_case(10, past_cases_dict)

20.656182567187603

In [61]:
time1 = time.time()

score_M = 0
for i in G.vs():
    score_M += calculate_score_for_case(i.index, past_cases_dict)
    
time2=time.time()
total_time = str(time2-time1)
print 'took ' + total_time
print
print score_M

UnboundLocalError: local variable 'all_past_cases' referenced before assignment